In [7]:
import jieba
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, glob, time, copy, random, zipfile
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.nn.functional as F
import torchvision
from torchvision import models, transforms

In [8]:
from os import listdir
from os.path import isfile, join,splitext
datapath = 'Stage_2\dataPublicComplete_s2\dataPublicComplete'
txt_fnames = [splitext(f)[0] for f in listdir(datapath) if isfile(join(datapath, f))]

In [9]:
crop_list = open('Keywords/02crop.list.csv', "r",encoding='UTF-8-sig')
crop = crop_list.read()
crop_line_sep = crop.splitlines()

pest_list = open('Keywords/02pest.list.csv', "r",encoding='UTF-8-sig')
pest = pest_list.read()
pest_line_sep = pest.splitlines()

chem_list = open('Keywords/02chem.list.csv', "r",encoding='UTF-8-sig')
chem = chem_list.read()
chem_line_sep = chem.splitlines()

In [10]:
from itertools import chain
vector_dict = {}
for idx,line in enumerate(chain(crop_line_sep,pest_line_sep,chem_line_sep)):
    l = line.split(',')
    for word in l:
        if(word == ''):continue
        jieba.add_word(word)
        vector_dict[word] = idx

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Renewrr\AppData\Local\Temp\jieba.cache
Loading model cost 0.703 seconds.
Prefix dict has been built successfully.


In [12]:
vectors = {}
for fname in txt_fnames:
    txt = open(datapath+'/'+fname+'.txt', "r",encoding="utf-8")
    content = txt.read()
    seg_list = jieba.cut(content, cut_all=True)
    vectors[fname] = [0]*764
    for seg in seg_list:
        if(seg in vector_dict):
            vectors[fname][vector_dict[seg]] = 1

In [15]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(2*764, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 2),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [83]:
net = NeuralNetwork()
net.load_state_dict(torch.load('best_checkpoint_last_b4000.pth'))
net.eval()
net = net.to(device)

In [84]:
out = []
with torch.no_grad():
    for fname1 in txt_fnames:
        for fname2 in txt_fnames:
            if(fname1 == fname2):continue
            else:
                comb_vec = torch.tensor(vectors[fname1]+vectors[fname2])
                comb_vec = comb_vec.type(torch.FloatTensor).to(device)
                lbl = net(comb_vec)
                out.append([(fname1,fname2),lbl.cpu()])


In [85]:
better = []
for o in out:
    if(abs(o[1][0]) < 0.0001):
        better.append(o[0])

In [86]:
print(len(better))

2640


In [87]:
for o in out:
    if(o[0] == ('1032','1036')):
        print(o)
out[100:200]

[('1032', '1036'), tensor([-0.0021, -0.0154])]


[[('1001', '1296'), tensor([-0.0058, -0.0151])],
 [('1001', '1299'), tensor([-0.0072, -0.0141])],
 [('1001', '13'), tensor([-0.0040, -0.0115])],
 [('1001', '130'), tensor([-0.0022, -0.0154])],
 [('1001', '1303'), tensor([-0.0023, -0.0164])],
 [('1001', '1306'), tensor([-0.0023, -0.0164])],
 [('1001', '1307'), tensor([-0.0023, -0.0164])],
 [('1001', '1313'), tensor([-0.0023, -0.0164])],
 [('1001', '1316'), tensor([-0.0072, -0.0141])],
 [('1001', '1317'), tensor([-0.0056, -0.0142])],
 [('1001', '1318'), tensor([-0.0065, -0.0150])],
 [('1001', '132'), tensor([-0.0056, -0.0167])],
 [('1001', '1325'), tensor([-0.0045, -0.0152])],
 [('1001', '1328'), tensor([-0.0040, -0.0178])],
 [('1001', '1330'), tensor([-0.0012, -0.0147])],
 [('1001', '1336'), tensor([-0.0047, -0.0130])],
 [('1001', '1338'), tensor([-0.0043, -0.0139])],
 [('1001', '134'), tensor([-0.0065, -0.0067])],
 [('1001', '1344'), tensor([-0.0062, -0.0170])],
 [('1001', '1345'), tensor([-0.0057, -0.0159])],
 [('1001', '1346'), tenso

In [77]:
import csv
with open('val.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(["Test"]+["Reference"])
    for row in better:  
        spamwriter.writerow(row)